# Задача

Вы работаете в компании *SkyJewel* (аналог Sokolov), которая занимается производством и продажей ювелирных изделий. 

В компании регулярно применяется рекламная акция, в соответствии с которой зарегистрированному в приложении пользователю приходит пуш-уведомление о выгодных скидках.

Было решено провести A/B Тест, где тестовой группе вместо пуш-уведомления приходит смс-сообщение с таким же содержанием.

Вам необходимо проанализировать результаты A/B Теста.


Скачайте файл для работы над заданием

[Домашняя работа Бизнес-3 (исходник).csv](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/7a637998-2cad-4000-bff5-320b7baae59e/Домашняя_работа_Бизнес-3_(исходник).csv)

**Описание таблицы:**
- ***id_client*** - уникальный идентификатор клиента
- ***id_group*** - идентификатор группы (0 - контроль, 1 - тест)
- ***city*** - название города
- ***nflag_purchase*** - факт покупки (1 - приобретен рекламируемый товар, 0 - нет)
- ***time_came*** - дата регистрации пользователя в приложении 
    

**Алгоритм выполнения задания:**
1. Рассчитайте результаты A/B Теста по всей совокупности и сделайте выводы.
2. Рассчитайте результаты A/B Теста для каждого города в отдельности и сделайте выводы.
3. Рассчитайте результаты A/B Теста для каждого города отдельно для тех, кто зарегистрировался в 2020 году, и отдельно для тех, кто зарегистрировался в 2021 году. Сделайте выводы.
4. (*) Подумайте о том, как можно автоматизировать алгоритм, чтобы не рассчитывать каждый раз одно и то же для каждого нового сегмента (ввести функцию с интерпретируемыми принтами).


---
# Установка библиотек, импорт библиотек и файлов

In [1]:
# импорт библиотек
import pandas as pd
from scipy import stats

In [2]:
# импорт файлов
file_name = 'Домашняя работа Бизнес-3 (исходник).csv'
data = pd.read_csv(file_name, sep=';')

# Решение

Посмотрим на данные и структуру таблицы

In [3]:
data.sample(10)

,id_client,id_group,city,nflag_purchase,time_came
541,24359894,0,Saint-Petersbourg,0,20.05.2020
3457,24357865,1,Novosibirsk,0,01.02.2021
4948,24358664,1,Moscow,1,02.09.2021
6124,24357491,0,Moscow,1,27.02.2020
1280,24358615,1,Saint-Petersbourg,0,22.01.2021
2014,24360975,1,Kazan,0,22.11.2020
540,24360314,1,Novosibirsk,0,31.01.2020
2179,24362941,0,Vladivostok,0,12.08.2021
4217,24358944,0,Novosibirsk,1,04.07.2020
5325,24360274,0,Saint-Petersbourg,0,16.07.2021


Приведем колонку даты к формату используемому в pandas

In [4]:
data['time_came'] = pd.to_datetime(data['time_came'], format='%d.%m.%Y')
data.sample(3)

,id_client,id_group,city,nflag_purchase,time_came
4186,24359784,0,Saint-Petersbourg,1,2021-05-10
5197,24361651,1,Kazan,0,2020-03-22
5085,24357297,1,Moscow,0,2021-03-28


Группируем данные для оценки

In [5]:
data_gr = data.groupby(['id_group', 'city']).agg({'nflag_purchase': 'sum', 'id_client': 'count'}).reset_index()
data_gr = data_gr.rename(columns={'nflag_purchase': 'sum_purchase', 'id_client': 'cnt_clients'})
data_gr['ratio'] = round(data_gr['sum_purchase'] / data_gr['cnt_clients'], 4)
data_gr.sort_values(by='city')

,id_group,city,sum_purchase,cnt_clients,ratio
0,0,Kazan,142,439,0.3235
5,1,Kazan,171,503,0.3400
1,0,Moscow,304,1106,0.2749
6,1,Moscow,311,1068,0.2912
2,0,Novosibirsk,152,585,0.2598
7,1,Novosibirsk,166,609,0.2726
3,0,Saint-Petersbourg,206,807,0.2553
8,1,Saint-Petersbourg,238,815,0.2920
4,0,Vladivostok,68,234,0.2906
9,1,Vladivostok,88,228,0.3860


### - 1. Рассчитайте результаты A/B Теста по всей совокупности и сделайте выводы.

In [6]:
# id_group - идентификатор группы (0 - контроль, 1 - тест)

cont_group = data.loc[data['id_group'] == 0, 'nflag_purchase']
test_group = data.loc[data['id_group'] == 1, 'nflag_purchase']

s, p = stats.ttest_ind(cont_group, test_group)
print(s, p)

-2.4013624185110922 0.01636252986305824


s = -2.4 Выходит за границы -1.96, 1.96, и p = 0.01 меньше 5% следовательно гипотеза Н0 отвергается, следовательно будем сравнивать гипотезы Н1 (тестовая группа лучше чем контрольная или наоборот) и определим результат А/В тестирования

In [7]:
cont_group.mean(), test_group.mean(), (test_group.mean() - cont_group.mean()) / test_group.mean()

(0.2749921160517187, 0.3022029165373875, 0.09004148867075011)

По результату расчета видим что среднее выборки тестовой группы выросла больше на 9% относительно контрольной группы

### - 2. Рассчитайте результаты A/B Теста для каждого города в отдельности и сделайте выводы.

In [8]:
def get_result_ABtest(test_df: pd.DataFrame, control_df: pd.DataFrame) -> None:
    '''Выводит на печать результат А/В тестирования.'''
    s, p = stats.ttest_ind(control_df, test_df)
    if (p > 0.05):
        print('Выборки не имеют существенной разницы!\nИзменения над тестовой группой не повлеяли на качественные показатели\nпо отношению к контрольной группуе...')
    else:
        print('РЕЗУЛЬТАТЫ ТЕСТИРОВАНИЯ:')
        if control_df.mean() > test_df.mean():
            diff = ((control_df.mean() - test_df.mean()) / control_df.mean()) * 100
            print(f'Средняя выборки тестовой группы по сравнению с контрольной УПАЛА на {round(diff, 2)}%!')
        elif control_df.mean() < test_df.mean():
            diff = ((test_df.mean() - control_df.mean()) / control_df.mean()) * 100
            print(f'Средняя выборки тестовой группы по сравнению с контрольной ВЫРОСЛА на {round(diff, 2)}%!')
    print('s = ', round(s, 2), '\np = ', round(p, 2),'\n')

In [9]:
for city in data['city'].unique():
    cont_group = data.loc[(data['id_group'] == 0) & (data['city'] == city), 'nflag_purchase']
    test_group = data.loc[(data['id_group'] == 1) & (data['city'] == city), 'nflag_purchase']
    print('------------------------------', city, '------------------------------')
    get_result_ABtest(test_group, cont_group)

------------------------------ Moscow ------------------------------
Выборки не имеют существенной разницы!
Изменения над тестовой группой не повлеяли на качественные показатели
по отношению к контрольной группуе...
s =  -0.85 
p =  0.4 

------------------------------ Saint-Petersbourg ------------------------------
Выборки не имеют существенной разницы!
Изменения над тестовой группой не повлеяли на качественные показатели
по отношению к контрольной группуе...
s =  -1.66 
p =  0.1 

------------------------------ Novosibirsk ------------------------------
Выборки не имеют существенной разницы!
Изменения над тестовой группой не повлеяли на качественные показатели
по отношению к контрольной группуе...
s =  -0.5 
p =  0.62 

------------------------------ Kazan ------------------------------
Выборки не имеют существенной разницы!
Изменения над тестовой группой не повлеяли на качественные показатели
по отношению к контрольной группуе...
s =  -0.54 
p =  0.59 

----------------------------

### - 3. Рассчитайте результаты A/B Теста для каждого города отдельно для тех, кто зарегистрировался в 2020 году, и отдельно для тех, кто зарегистрировался в 2021 году. Сделайте выводы.

In [10]:

dct_dfs = {
    '2020': data[data['time_came'].dt.year == 2020],
    '2021': data[data['time_came'].dt.year == 2021]
}

for year, df in dct_dfs.items():
    print('==============================================================================')
    print('Оценка по',year, 'году\n')
    
    for city in df['city'].unique():
        cont_group = df.loc[(df['id_group'] == 0) & (df['city'] == city), 'nflag_purchase']
        test_group = df.loc[(df['id_group'] == 1) & (df['city'] == city), 'nflag_purchase']
        print('------------------------------', city, year, '------------------------------')
        get_result_ABtest(test_group, cont_group)

Оценка по 2020 году

------------------------------ Saint-Petersbourg 2020 ------------------------------
Выборки не имеют существенной разницы!
Изменения над тестовой группой не повлеяли на качественные показатели
по отношению к контрольной группуе...
s =  -1.06 
p =  0.29 

------------------------------ Kazan 2020 ------------------------------
Выборки не имеют существенной разницы!
Изменения над тестовой группой не повлеяли на качественные показатели
по отношению к контрольной группуе...
s =  -0.61 
p =  0.54 

------------------------------ Moscow 2020 ------------------------------
Выборки не имеют существенной разницы!
Изменения над тестовой группой не повлеяли на качественные показатели
по отношению к контрольной группуе...
s =  -0.67 
p =  0.51 

------------------------------ Vladivostok 2020 ------------------------------
РЕЗУЛЬТАТЫ ТЕСТИРОВАНИЯ:
Средняя выборки тестовой группы по сравнению с контрольной ВЫРОСЛА на 44.46%!
s =  -2.95 
p =  0.0 

-----------------------------

### - 4. (*) Подумайте о том, как можно автоматизировать алгоритм, чтобы не рассчитывать каждый раз одно и то же для каждого нового сегмента (ввести функцию с интерпретируемыми принтами).


*Мы уже применили одну функцию при оценки А/В теста (get_result_ABtest), предлагаю вазять ее за основу*

In [11]:
# <1> ======================================================================================================================

def get_result_ABtest(test_df: pd.DataFrame, control_df: pd.DataFrame) -> None:
    '''
    Выводит на печать результат А/В тестирования.
    
    :param test_df: Выборка тестовой группы параметров
    :param control_df: Выборка генеральной группы параметров
    '''
    s, p = stats.ttest_ind(control_df, test_df)
    if (p > 0.05):
        print('Выборки не имеют существенной разницы!\nИзменения над тестовой группой не повлеяли на качественные показатели\nпо отношению к контрольной группуе...')
    else:
        print('РЕЗУЛЬТАТЫ ТЕСТИРОВАНИЯ:')
        if control_df.mean() > test_df.mean():
            diff = ((control_df.mean() - test_df.mean()) / control_df.mean()) * 100
            print(f'Средняя выборки тестовой группы по сравнению с контрольной УПАЛА на {round(diff, 2)}%!')
        elif control_df.mean() < test_df.mean():
            diff = ((test_df.mean() - control_df.mean()) / control_df.mean()) * 100
            print(f'Средняя выборки тестовой группы по сравнению с контрольной ВЫРОСЛА на {round(diff, 2)}%!')
    print('s = ', round(s, 2), '\np = ', round(p, 2),'\n')
    
# <2> ======================================================================================================================
    
    
def result_AB_test(df: pd.DataFrame, name_col_testing: str, name_col_flag: str, group_by: str=None) -> None:
    '''
    Получает DataFrame для оценки результатов А/В тестирования и выводит на печать результат.
    
    :param df: Данные для оценки
    :param name_col_flag: Имя колонки, по которой отличается генеральная выборка от тестовой (0 - контроль, 1 - тест)
    :param group_by: Колонка для группировки, если требуется (при групповой оценки тестирования)
    '''
    if group_by == None:
        get_result_ABtest(df.loc[df[name_col_flag] == 0, name_col_testing], df.loc[df[name_col_flag] == 1, name_col_testing])
    else:    
        for meaning in df[group_by].unique():
            cont_group = df.loc[(df[name_col_flag] == 0) & (df[group_by] == meaning), name_col_testing]
            test_group = df.loc[(df[name_col_flag] == 1) & (df[group_by] == meaning), name_col_testing]
            print('------------------------------', meaning, '------------------------------')
            get_result_ABtest(test_group, cont_group)

**Тестируем работу функции:**

In [12]:
data.sample(3)

,id_client,id_group,city,nflag_purchase,time_came
4476,24363068,1,Vladivostok,1,2021-05-29
305,24357375,0,Moscow,1,2021-08-09
3652,24359367,1,Moscow,0,2021-05-03


In [13]:
# Рассчитайте результаты A/B Теста по всей совокупности
result_AB_test(data, 'nflag_purchase', 'id_group')

РЕЗУЛЬТАТЫ ТЕСТИРОВАНИЯ:
Средняя выборки тестовой группы по сравнению с контрольной УПАЛА на 9.0%!
s =  2.4 
p =  0.02 



In [14]:
# Рассчитайте результаты A/B Теста по каждому городу
result_AB_test(data, 'nflag_purchase', 'id_group', 'city')

------------------------------ Moscow ------------------------------
Выборки не имеют существенной разницы!
Изменения над тестовой группой не повлеяли на качественные показатели
по отношению к контрольной группуе...
s =  -0.85 
p =  0.4 

------------------------------ Saint-Petersbourg ------------------------------
Выборки не имеют существенной разницы!
Изменения над тестовой группой не повлеяли на качественные показатели
по отношению к контрольной группуе...
s =  -1.66 
p =  0.1 

------------------------------ Novosibirsk ------------------------------
Выборки не имеют существенной разницы!
Изменения над тестовой группой не повлеяли на качественные показатели
по отношению к контрольной группуе...
s =  -0.5 
p =  0.62 

------------------------------ Kazan ------------------------------
Выборки не имеют существенной разницы!
Изменения над тестовой группой не повлеяли на качественные показатели
по отношению к контрольной группуе...
s =  -0.54 
p =  0.59 

----------------------------